## Piplines
Le **pipeline** sono un modo semplice per organizzare il codice di preelaborazione e modellazione dei dati. In particolare, una **pipeline**raggruppa le fasi di preelaborazione e modellazione in modo da poter utilizzare l'intero pacchetto come se fosse un'unica fase.

Molti scienziati dei dati mettono insieme i modelli senza ricorrere alle pipeline, ma queste ultime presentano alcuni vantaggi importanti. Tra questi:

1. Codice più pulito: La gestione dei dati in ogni fase della preelaborazione può essere disordinata. Con una pipeline, non sarà necessario tenere traccia manualmente dei dati di addestramento e di convalida in ogni fase.
2. Meno bug: Ci sono meno opportunità di applicare erroneamente una fase o di dimenticare una fase di preelaborazione.
3. Più facile da produrre: Può essere sorprendentemente difficile passare un modello da un prototipo a qualcosa di distribuibile su scala. Non ci soffermeremo qui sui numerosi problemi correlati, ma le pipeline possono essere d'aiuto.
4. Più opzioni per la convalida del modello: Vedrete un esempio nel prossimo tutorial, che tratta la convalida incrociata.

### Esempio
Come nel tutorial precedente, lavoreremo con il dataset Melbourne Housing.

Non ci concentreremo sulla fase di caricamento dei dati. Si può invece immaginare di trovarsi a un punto in cui i dati di addestramento e di validazione sono già presenti in `X_train`, `X_valid`, `y_train` e `y_valid`.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('D:/Users/Alessio/OneDrive/Python/Kaggle/Intermediate Machine Learning/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Diamo un'occhiata ai dati di allenamento con il metodo `head()`. Si noti che i dati contengono sia dati categorici che colonne con valori mancanti. Con una **pipeline**, è facile gestire entrambi.

In [2]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


### Pipeline in tre fasi:

1. **Definire le fasi di preelaborazione**   
Analogamente a come una pipeline raggruppa le fasi di preelaborazione e modellazione, utilizziamo la classe ColumnTransformer per raggruppare diverse fasi di preelaborazione. Il codice seguente:

- imputa i valori mancanti nei dati numerici 
- imputa i valori mancanti e applica una codifica one-hot ai dati categorici.

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

2. **Definire il modello**  
Definiamo quindi un modello di foresta casuale con la nota classe `RandomForestRegressor`.

In [4]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

3. **Creare e valutare la pipeline**  
Infine, usiamo la classe **Pipeline** per definire una pipeline che raggruppa le fasi di preelaborazione e modellazione. Ci sono alcune cose importanti da notare:

- Con la **pipeline**, preelaboriamo i dati di addestramento e adattiamo il modello in un'unica riga di codice. (Al contrario, senza una pipeline, dobbiamo eseguire l'**imputazione**, la **codifica one-hot** e l'**addestramento del modello** in fasi separate. Questo diventa particolarmente complicato se dobbiamo gestire variabili sia numeriche che categoriche).
- Con la **pipeline**, forniamo le caratteristiche non elaborate in `X_valid` al comando `predict()` e la pipeline preelabora automaticamente le caratteristiche prima di generare le previsioni. (Tuttavia, senza una **pipeline**, dobbiamo ricordarci di preelaborare i dati di validazione prima di fare le previsioni).

In [5]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855


Le **pipeline** sono preziose per ripulire il codice di apprendimento automatico ed evitare errori, e sono particolarmente utili per i flussi di lavoro con una sofisticata preelaborazione dei dati.

## Esercizi
In questo esercizio si utilizzeranno le **pipeline** per migliorare l'efficienza del codice di apprendimento automatico.

Le domande che seguono vi daranno un feedback sul vostro lavoro. Eseguire la seguente cella per impostare il sistema di feedback.

In [7]:
# Set up code checking
import os
if not os.path.exists("D:/Users/Alessio/OneDrive/Python/Kaggle/Intermediate Machine Learning/train.csv"):
    os.symlink("D:/Users/Alessio/OneDrive/Python/Kaggle/Intermediate Machine Learning/train.csv")  
    os.symlink("D:/Users/Alessio/OneDrive/Python/Kaggle/Intermediate Machine Learning/test.csv") 
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex4 import *
print("Setup Complete")

Setup Complete


Eseguire la cella di codice successiva senza modifiche per caricare gli insiemi di addestramento e validazione in `X_train`, `X_valid`, `y_train` e `y_valid`.  L'insieme di test viene caricato in `X_test`.

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('D:/Users/Alessio/OneDrive/Python/Kaggle/Intermediate Machine Learning/train.csv', index_col='Id')
X_test_full = pd.read_csv('D:/Users/Alessio/OneDrive/Python/Kaggle/Intermediate Machine Learning/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [9]:
X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


La cella di codice successiva utilizza il codice dell'esercitazione per preelaborare i dati e addestrare un modello. Eseguire questo codice senza modifiche.

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17861.780102739725


Il codice produce un valore di circa 17862 per l'errore assoluto medio (MAE).  Nella fase successiva, modificherete il codice per ottenere risultati migliori.

### Domanda 1
**Migliorare le prestazioni**

Nella cella di codice sottostante, definite le vostre fasi di preelaborazione e il modello di foresta casuale. Inserite i valori delle seguenti variabili:
- `numerical_transformer`
- `categorical_transformer`
- `model`

Per superare questa parte dell'esercizio, è sufficiente definire passaggi di preelaborazione validi e un modello di foresta casuale.

In [11]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Check your answer
step_1.a.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

Eseguire la cella di codice sottostante senza modifiche.

Per superare questa fase, è necessario aver definito una pipeline nella **Parte A** che raggiunga un MAE inferiore a quello del codice precedente.  Vi invitiamo a prendere tempo e a provare diversi approcci, per vedere quanto riuscite a ridurre il MAE!  (_Se il vostro codice non passa, modificate le fasi di preelaborazione e il modello._)

In [12]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

# Check your answer
step_1.b.check()

MAE: 17621.3197260274


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### Domanda 2
**Generare previsioni di test**

Ora si utilizzerà il modello addestrato per generare previsioni con i dati di test.

In [13]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test)

# Check your answer
step_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

Eseguire la cella di codice successiva senza modifiche per salvare i risultati in un file CSV da inviare direttamente al concorso.

In [14]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)